In [15]:
import numpy as np
import math
#import pymeshlab
#import potpourri3d as pp3d
import polyscope as ps
from barmesh.tribarmes import trianglebarmesh
from barmesh.basicgeo import P3

In [16]:
#ms = pymeshlab.MeshSet()
#fname = 'geometry/simple_geom.stl'
#fname = 'geometry/spot_triangulated.stl'
#fname = 'geometry/duct.stl'
fname = 'geometry/PPR.stl'
#ms.load_new_mesh(fname)
#V, F = pp3d.read_mesh(fname)
tbm = trianglebarmesh.TriangleBarMesh(fname)
#n = ms.current_mesh().face_normal_matrix()
V = tbm.GetVertices()
F = tbm.GetFaces()
n = tbm.GetNormals()

In [17]:
def pointOnEdge(i, l):
    pt1 = tbm.bars[i].GetNodeFore(True).p
    pt2 = tbm.bars[i].GetNodeFore(False).p
    vec12 = (pt2[0]-pt1[0],pt2[1]-pt1[1],pt2[2]-pt1[2])
    ptl = P3(pt1[0]+l*vec12[0],pt1[1]+l*vec12[1],pt1[2]+l*vec12[2])
    return ptl

def P3list2array(P3list):
    pts =[]
    for p in P3list:
        pts.append((p.x,p.y,p.z))
    return np.array(pts)


def find_rot(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """

    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

def find_intersection(p1,vec1,p2,vec2, tol= 1e-6):
    flip = vec2.y ==0
    if flip:
        p1,vec1,p2,vec2 = p2,vec2,p1,vec1
    a = (p2.x+(vec2.x*p1.y-vec2.x*p2.y)/vec2.y-p1.x) / (vec1.x - vec2.x*vec1.y/vec2.y)
    b = (p1.y+ a*vec1.y-p2.y)/vec2.y
    p_int1 = P3(p1.x+ a*vec1.x,p1.y+ a*vec1.y,p1.z+ a*vec1.z)
    #print('tol ',abs((p1.z+ a*vec1.z)-(p2.z+ b*vec2.z)))
    assert abs((p1.z+ a*vec1.z)-(p2.z+ b*vec2.z)) < tol, "Lines do not intersect in 3D space"    
    if flip:
        a = b
    return p_int1,a

#function to rotate a P3 vector about an axis (of any length) and an angle in degrees
def rotAxisAngle(v,ax,a):
    ax = P3.ZNorm(ax)
    c = np.cos(np.radians(a))
    s = np.sin(np.radians(a))
    C = 1 - np.cos(np.radians(a))
    Q= np.array(((ax.x*ax.x*C+c,      ax.x*ax.y*C-ax.z*s, ax.x*ax.z*C+ax.y*s),
                 (ax.y*ax.x*C+ax.z*s, ax.y*ax.y*C+c,      ax.y*ax.z*C-ax.x*s),
                 (ax.z*ax.x*C-ax.y*s, ax.z*ax.y*C+ax.x*s, ax.z*ax.z*C+c)))
    v = np.dot(Q,np.array((v.x,v.y,v.z)))
    return P3(v[0],v[1],v[2])
    


In [18]:
# axis is a b
# incoming at d = Along(lam, a, b)
# incoming from c to d 
# incoming (d - c).(b - a) = |d-c||b-a|cos(theta) = |d-c|I
# I = |b-a|cos(theta) = (d - c).(b - a)/|d-c|

# triangle on right hand side is to e
# outgoing to x where x = Along(q, a, e) or x = Along(q, b, e)
# Solve (x - d).(b - a) = |x-d||b-a|cos(theta) = |x-d|I

# Set d = 0, v = b - a
# x.v = |x|I  where x = a + (e - a)*q = a + f*q
# x.x I^2 = (x.v)^2

# x.x I^2 = (a + f*q)^2 I^2 = q^2 f^2 I^2 + 2q a.f I^2 + a^2 I^2
#   =
# (x.v)^2 = (a.v + f.v*q)^2 = q^2 (f.v)^2 + 2q (a.v)(f.v) + (a.v)^2
# q^2 ((f.v)^2 - f^2 I^2) + 2q ((a.v)(f.v) - a.f I^2) + (a.v)^2 - a^2 I^2

def Square(X):
    return X*X
def TOL_ZERO(X):
    if not (abs(X) < 0.0001):
        print("TOL_ZERO fail", X)

def GeoCrossAxisE(a, Vae, Vab, Isq, Isgn):
    # Solve: Isq*x.Lensq() - Square(P3.Dot(x, Vab)) = 0   for x = a + Vae*q
    # 0 = Isq*(a^2 + 2q a.Vae + q^2 Vae^2) - (a.Vab + Vae.Vab q)^2
    #   = Isq*(a^2 + 2q adf + q^2 Vae^2) - (adv + fdv q)^2

    fdv = P3.Dot(Vae, Vab)
    adv = P3.Dot(a, Vab)
    adf = P3.Dot(a, Vae)
    qA = Square(fdv) - Vae.Lensq()*Isq
    qB2 = adv*fdv - adf*Isq
    qC = Square(adv) - a.Lensq()*Isq
    if abs(qA) <= abs(qB2)*1e-7:
        if qB2 == 0:
            return -1.0
        q = -qC/(2*qB2)
    else:
        qdq = Square(qB2) - qA*qC
        if qdq < 0.0:
            #print("qdq", qdq)
            return -1.0
        qs = math.sqrt(qdq) / qA
        qm = -qB2 / qA
        q = qm + qs*Isgn
    # q = qs +- qm,  x = a + Vae*q,  Dot(x, Vab) same sign as Dot(Vcd, Vab)
    if abs(q) < 100:
        TOL_ZERO(qA*Square(q) + qB2*2*q + qC)
    return q

#
# This is the basic function that crosses from one triangle to the next
#
def GeoCrossAxis(Ga, Gb, Gc, lam, Ge):
    Vab = Gb - Ga
    Gd = Ga + Vab*lam
    Vcd = Gd - Gc
    cdDab = P3.Dot(Vcd, Vab)
    Isq = Square(cdDab) / Vcd.Lensq()
    Isgn = -1 if cdDab < 0 else 1
    qVae = GeoCrossAxisE(Ga - Gd, Ge - Ga, Vab, Isq, Isgn)
    qVbe = GeoCrossAxisE(Gb - Gd, Ge - Gb, -Vab, Isq, -Isgn)
    bAEcrossing = (abs(qVae - 0.5) < abs(qVbe - 0.5))
    q = qVae if bAEcrossing else qVbe
    Gx = (Ga + (Ge - Ga)*q) if bAEcrossing else (Gb + (Ge - Gb)*q)
    Dx = Gx - Gd
    TOL_ZERO(Isq - Square(P3.Dot(Dx, Vab)/Dx.Len()))
    TOL_ZERO(P3.Dot(Vcd, Vab)/Vcd.Len() - P3.Dot(Dx, Vab)/Dx.Len())
    return bAEcrossing, q, Gx

In [19]:
# This is the iterative function that goes from bar to bar through the mesh
# c=from point, bar the crossing edge, lam the point on the bar, and bGoRight is the crossing direction
#
def GeoCrossBar(c, bar, lam, bGoRight, bPrintvals=False):
    bEnd = False
    Na, Nb = bar.nodeback, bar.nodefore
    if bGoRight:
        if bar.barforeright == None:
            bEnd = True
            Ne = trianglebarmesh.TriangleNode(P3(bar.nodefore.p.x+0.01,bar.nodefore.p.y+0.01,bar.nodefore.p.z+0.01),-1)
        else:
            Ne = bar.barforeright.GetNodeFore(bar.barforeright.nodeback == bar.nodefore)
    else:
        if bar.barbackleft == None:
            bEnd = True
            Ne = trianglebarmesh.TriangleNode(P3(bar.nodeback.p.x+0.01,bar.nodeback.p.y+0.01,bar.nodeback.p.z+0.01),-1)
        else:
            Ne = bar.barbackleft.GetNodeFore(bar.barbackleft.nodeback == bar.nodeback)
    d = Na.p + (Nb.p - Na.p)*lam
    if bPrintvals:
        print("\n", (Na.p, Nb.p, c, lam, Ne.p))
    bAEcrossing, q, Gx = GeoCrossAxis(Na.p, Nb.p, c, lam, Ne.p)
    if bGoRight:
        if bAEcrossing:
            if not bEnd:
                bar = bar.barforeright.GetForeRightBL(bar.barforeright.nodeback == Nb)
                lam = q if bar.nodeback == Na else 1-q
                bGoRight = (bar.nodeback == Na)
            else:
                bar = None
        else:
            if not bEnd:
                bar = bar.barforeright
                lam = q if bar.nodeback == Nb else 1-q
                bGoRight = not (bar.nodeback == Nb)
            else:
                bar = None
    else:
        if bAEcrossing:
            if not bEnd:
                bar = bar.barbackleft
                lam = q if bar.nodeback == Na else 1-q
                bGoRight = not (bar.nodeback == Na)
            else:
                bar = None
        else:
            if not bEnd:
                bar = bar.barbackleft.GetForeRightBL(bar.barbackleft.nodeback == Na)
                lam = q if bar.nodeback == Nb else 1-q
                bGoRight = (bar.nodeback == Nb)
            else:
                bar = None
    if not bEnd:
        c = bar.nodeback.p + (bar.nodefore.p - bar.nodeback.p)*lam
        TOL_ZERO((c - Gx).Len())
    else:
        c = None
    if bPrintvals:
        print("d,c", (d, c))
    
    return (d, bar, lam, bGoRight)



In [20]:
#Draw a single geodesic line from a specified point at a specified angle and ref90 A reference direction approximately perpendicular to the axis of the part
    
def createGeoLine(startPt, startFace, theta, ref90, forward = True, maxPathLength = 1000):
    valid = True
    fail = 0
    ref0 = P3.Cross(ref90,tbm.faces[startFace].normal)
    startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
    
    #find which bar intersects with the start vector
    barsint = []
    for b in tbm.faces[startFace].bars:
        p1 = tbm.bars[b].GetNodeFore(True).p
        vec1 = tbm.bars[b].GetNodeFore(False).p-tbm.bars[b].GetNodeFore(True).p
        try:
            Pint, lam = find_intersection(p1,vec1,startPt,startVec,tol=1e-4)
        except:
            lam = -1
            print('Error finding instersection between starting lines and edge of face, try specifying ref90 in different plane')
        if 0 < lam < 1:
            barsint.append((b,1-lam))
    bar = tbm.bars[barsint[int(forward)][0]]
    bGoRight = bar.faceleft == startFace
    lam = barsint[int(forward)][1]
    
    pts = [startPt]
    faces = [startFace]
    curvs = [0]
    finished = False
    bar_last = 0
    while not finished:
        if bar != None:
            if bGoRight:
                faces.append(bar.faceright)
            else:
                faces.append(bar.faceleft)
            bar_last = bar.i
            c, bar, lam, bGoRight = GeoCrossBar(pts[-1], bar, lam, bGoRight)
            pts.append(c)
            if faces[-1] != None:
                CClast = pts[-1]-pts[-2]
                curvature = P3.Dot(tbm.faces[faces[-1]].normal,CClast)/P3.Len(tbm.faces[faces[-1]].normal)
                curvs.append(curvs[-1]*0.75 + curvature*0.25)
            elif faces[-1] == None:
                curvs.append(curvs[-1])
        
        if bar==None:
            finished =True
            print('edge reached on path', theta)
            for be in badEdgeBars:
                if bar_last == be:
                    print('PATH FAIL: bad edge reached')
                    valid = False
                    fail = 1
        elif len(pts)>maxPathLength:
            finished =True
            print('max path length reached on path', theta)
        #elif ## SOME FUNCTION TO DETERMINE IF CURVATURE HAS BEEN EXCEEDED AND SET FAIL TO 2
        else:
            finished =False
    return {'pts':pts,'curvs':curvs,'faces':faces, 'valid':valid, 'fail':fail}

def createDoubleGeoLine(startPt, startFace, theta, ref90, maxPathLength = 1000):
    geoline = createGeoLine(startPt, startFace, theta, ref90, True, maxPathLength)
    geolineB = createGeoLine(startPt, startFace, theta, ref90, False, maxPathLength)
    for i in range(1,len(geolineB['pts'])):
        geoline['pts'].insert(0,geolineB['pts'][i])
        geoline['faces'].insert(0,geolineB['faces'][i])
        geoline['curvs'].insert(0,geolineB['curvs'][i])
        geoline['valid'] = geoline['valid'] and geolineB['valid']
        geoline['fail'] = [geoline['fail'],geolineB['fail']]
    return geoline

In [21]:
geopaths=[]
curvs = []
faces =[]
barF,bGoRightF,lamF,barB,bGoRightB,lamB = [],[],[],[],[],[]
select = 0
thetas = ()
maxPathLength = 1000
badFaces = []
edgeBars = tbm.GetEdgeBars()
badEdgeBars = []
geolines =[]

def init_path(startFace):
    global geopaths
    global curvs
    global faces
    global barF,bGoRightF,lamF,barB,bGoRightB,lamB
    global thetas
    geopaths=[]
    curvs = []
    faces =[]
    barF,bGoRightF,lamF,barB,bGoRightB,lamB = [],[],[],[],[],[]
    x,y,z = 0,0,0
    for nn in tbm.faces[startFace].nodes:
        x += tbm.nodes[nn].p.x/3
        y += tbm.nodes[nn].p.y/3
        z += tbm.nodes[nn].p.z/3

    startPt = P3(x,y,z)#
    ref90 = P3(1,0,0)# A reference direction approximately perpendicular to the axis of the part
    ref0 = P3.Cross(ref90,tbm.faces[startFace].normal)
    thetas = np.arange(0,180,10)

    barsints =[]
    for theta in thetas:
        startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
        barsint =[]
        #find which bars intersect with the start vector
        for b in tbm.faces[startFace].bars:
            p1 = tbm.bars[b].GetNodeFore(True).p
            vec1 = tbm.bars[b].GetNodeFore(False).p-tbm.bars[b].GetNodeFore(True).p
            try:
                Pint, lam = find_intersection(p1,vec1,startPt,startVec,tol=1e-4)
            except:
                lam = -1
                print('Error finding instersection between starting lines and edge of face, try specifying ref90 in different plane')
            if 0 < lam < 1:
                barsint.append((b,1-lam))
        barsints.append(barsint)

    for barsint in barsints:
        barF.append(tbm.bars[barsint[0][0]])
        bGoRightF.append(barF[-1].faceleft == startFace)
        lamF.append(barsint[0][1])
        barB.append(tbm.bars[barsint[1][0]])
        bGoRightB.append(barB[-1].faceleft == startFace)
        lamB.append(barsint[1][1])
        geopaths.append([startPt])
        curvs.append([0])
        faces.append([startFace])
        

def draw_paths():
    global geolines
    global select
    global badEdgeBars
    thetas = np.arange(0,180,10)
    if ps.get_selection()[1] != select:
        select = ps.get_selection()[1]
        if len(V) < select < (len(V)+len(F)):
            fselect = select - len(V)
            print('face',fselect)
        else:
            return
    else:
        return
    x,y,z = 0,0,0
    for nn in tbm.faces[fselect].nodes:
        x += tbm.nodes[nn].p.x/3
        y += tbm.nodes[nn].p.y/3
        z += tbm.nodes[nn].p.z/3

    startPt = P3(x,y,z)
    ref90 = P3(1,0,0)
    geolines =[]
    for th in thetas:
        print('calculating',th)
        geolineDouble = createDoubleGeoLine(startPt, fselect, th, ref90)
        geolines.append(geolineDouble)
                
    for i in range(len(geolines)):
        if len(geolines[i]['pts'])>0:
            ps_geopath = ps.register_curve_network(str(thetas[i])+'deg', P3list2array(geolines[i]['pts']), 'line')
            if len(geolines[i]['curvs']) == len(geolines[i]['pts']):
                ps_geopath.add_scalar_quantity("curvature", np.array(geolines[i]['curvs']), defined_on='nodes', enabled=True,  cmap='coolwarm',vminmax=(-0.005, 0.005))  
                

def increment_path():
    global geopaths
    global curvs
    global faces
    global barF,bGoRightF,lamF,barB,bGoRightB,lamB
    global select
    global thetas
    if ps.get_selection()[1] != select:
        select = ps.get_selection()[1]
        if len(V) < select < (len(V)+len(F)):
            fselect = select - len(V)
            print('face',fselect)
            init_path(fselect)
        
    allfinished = True
    for i in range(len(thetas)):
        if barF[i]==None and barB[i] ==None:
            f=True
        else:
            f =False
        allfinished = allfinished and f
    
    for i in range(len(thetas)):
        if allfinished: break
        if barF[i] != None:
            if bGoRightF[i]:
                faces[i].append(barF[i].faceright)
            else:
                faces[i].append(barF[i].faceleft)
            #print('face: ',faces[-1])
            c, barF[i], lamF[i], bGoRightF[i] = GeoCrossBar(geopaths[i][-1], barF[i], lamF[i], bGoRightF[i])
            geopaths[i].append(c)
            if faces[i][-1] != None:
                CClast = geopaths[i][-1]-geopaths[i][-2]
                curvature = P3.Dot(tbm.faces[faces[i][-1]].normal,CClast)/P3.Len(tbm.faces[faces[i][-1]].normal)
                curvs[i].append(curvs[i][-1]*0.75 + curvature*0.25)
            elif faces[i][-1] == None:
                curvs[i].append(curvs[i][-1])

        if barB[i] != None:
            if bGoRightB[i]:
                faces[i].insert(0,barB[i].faceright)
            else:
                faces[i].insert(0,barB[i].faceleft)
            #print('face: ',faces[-1])
            c, barB[i], lamB[i], bGoRightB[i] = GeoCrossBar(geopaths[i][0], barB[i], lamB[i], bGoRightB[i])
            geopaths[i].insert(0,c)
            if faces[i][0] != None:
                CClast = geopaths[i][0]-geopaths[i][1]
                curvature = P3.Dot(tbm.faces[faces[i][0]].normal,CClast)/P3.Len(tbm.faces[faces[i][0]].normal)
                curvs[i].insert(0,curvs[i][0]*0.75 + curvature*0.25)
            elif faces[i][0] == None:
                curvs[i].insert(0,curvs[i][0])
    for i in range(len(geopaths)):
        if allfinished: break
        ps_geopath = ps.register_curve_network(str(thetas[i])+'deg', P3list2array(geopaths[i]), 'line')
        if len(curvs[i]) == len(geopaths[i]):
            ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='nodes', enabled=True,  cmap='coolwarm',vminmax=(-0.005, 0.005))

        
def selectBadFaces():
    global badFaces
    global select
    global edgeBars
    global badEdgeBars
    if ps.get_selection()[1] != select:
        select = ps.get_selection()[1]
        if len(V) < select < (len(V)+len(F)):
            fselect = select - len(V)
            print('face',fselect)
            badFaces.append(fselect)
            for b in tbm.faces[fselect].bars:
                for eb in edgeBars:
                    if b == eb:
                        badEdgeBars.append(eb)
            col =[]
            B=(0.1,0.1,1)
            R=(1,0.1,0.1)
            for f in tbm.faces:
                bad = False
                for bf in badFaces:
                    if bf == f.i:
                        col.append(R)
                        bad = True
                if not bad:
                    col.append(B)
            #ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
            ps_surf.add_color_quantity("bad faces", np.array(col), defined_on='faces',enabled = True)
            

    else:
        return
            


In [22]:
ps.init()

ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)

ps.set_user_callback(selectBadFaces)
ps.show()
ps.clear_user_callback()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.6 (Core Profile) Mesa 21.2.6
face 7803
face 7795
face 9297
face 7780
face 9544
face 8862
face 8717
face 8738
face 8940
face 7517
face 8307
face 7804
face 8303
face 9302
face 8603
face 9303
face 8302
face 7802
face 8305
face 7515
face 8939
face 8736
face 8716
face 8861
face 9543
face 7779
face 9296
face 7796
face 7801
face 8304
face 7514
face 8938
face 8737
face 8695
face 7888
face 8349
face 7589
face 8350
face 7887
face 8694
face 8739
face 8941
face 7516
face 8306


In [23]:
def findGoodEdge(edgeBars,badEdgeBars):
    goodEdgeBars =[]
    for eb in edgeBars:
        good = True
        for beb in badEdgeBars:
            if eb == beb: good = False
        if good: goodEdgeBars.append(eb)
    return goodEdgeBars

def sortBars(tbm, unsortedBars):
    sortedBars = []
    eb1 = unsortedBars[0]
    
    for i in range(len(unsortedBars)):
        for eb2 in unsortedBars:
            if tbm.bars[eb1].GetNodeFore(True) == tbm.bars[eb2].GetNodeFore(True) or tbm.bars[eb1].GetNodeFore(True) == tbm.bars[eb2].GetNodeFore(False) or tbm.bars[eb1].GetNodeFore(False) == tbm.bars[eb2].GetNodeFore(True) or tbm.bars[eb1].GetNodeFore(False) == tbm.bars[eb2].GetNodeFore(False):
                print('match found at',eb2)
                sortedBars.append(eb2)
                unsortedBars.remove(eb2)
                eb1 = eb2
                break

    return sortedBars

goodEdgeBars = findGoodEdge(edgeBars,badEdgeBars)
goodEdgeBars = sortBars(tbm, goodEdgeBars)

#startPts =[tbm.bars[goodEdgeBars[0]].GetNodeFore(True).p]
seeds = {'pts':[],'faces':[],'ref90s':[],'bFor':[]}
for b in goodEdgeBars:
#    if tbm.bars[b].GetNodeFore(True).p == startPts[-1]:
#        p = tbm.bars[b].GetNodeFore(False).p
#    else:
#        p =tbm.bars[b].GetNodeFore(True).p
#    startPts.append(p)
    x = (tbm.bars[b].GetNodeFore(True).p.x + tbm.bars[b].GetNodeFore(False).p.x)/2
    y = (tbm.bars[b].GetNodeFore(True).p.y + tbm.bars[b].GetNodeFore(False).p.y)/2
    z = (tbm.bars[b].GetNodeFore(True).p.z + tbm.bars[b].GetNodeFore(False).p.z)/2
    p = P3(x,y,z)
    seeds['pts'].append(p)    
    
    if tbm.bars[b].faceright:
        seeds['faces'].append(tbm.bars[b].faceright)
        seeds['bFor'].append(False)
        v = tbm.bars[b].GetNodeFore(True).p - tbm.bars[b].GetNodeFore(False).p
    else:
        seeds['faces'].append(tbm.bars[b].faceleft)
        seeds['bFor'].append(True)
        v = tbm.bars[b].GetNodeFore(False).p - tbm.bars[b].GetNodeFore(True).p
    seeds['ref90s'].append(v)

match found at 2
match found at 3
match found at 254
match found at 446
match found at 639
match found at 831
match found at 1023
match found at 1225
match found at 1438
match found at 1639
match found at 1864
match found at 2087
match found at 2331
match found at 2569
match found at 2805
match found at 3062
match found at 3323
match found at 3588
match found at 3865
match found at 4153
match found at 4443
match found at 4745
match found at 5052
match found at 5356
match found at 5658
match found at 5932
match found at 6224
match found at 6500
match found at 6762
match found at 7037
match found at 7330
match found at 7609
match found at 7915
match found at 8188
match found at 8489
match found at 8779
match found at 9097
match found at 9385
match found at 9722
match found at 10029
match found at 10334
match found at 10626
match found at 10941
match found at 11237
match found at 11499
match found at 11746
match found at 12013
match found at 12247
match found at 12506
match found at 12766

In [24]:
theta = 22
geolines=[]
ref0s =[]
ref90s = []
startPts =[]
startVecs =[]
for i in range(len(seeds['pts'])):
    startPt = seeds['pts'][i]
    startPts.append(startPt)
    ref90 = seeds['ref90s'][i]
    ref90s.append(ref90)
    startFace =seeds['faces'][i]
    ref0 = P3.Cross(ref90,tbm.faces[startFace].normal)
    ref0s.append(ref0)
    startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
    startVecs.append(startVec)
    forward = seeds['bFor'][i]
    geoline = createGeoLine(startPt, startFace, theta, ref90, forward, maxPathLength = 1000)
    if len(geoline['pts']) < 3:
        geoline = createGeoLine(startPt, startFace, theta, ref90, not forward, maxPathLength = 1000)
    geolines.append(geoline)

edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on path 22
edge reached on 

In [25]:
theta = 45
geolines2=[]

for i in range(len(seeds['pts'])):
    startPt = seeds['pts'][i]
    ref90 = seeds['ref90s'][i]
    startFace =seeds['faces'][i]
    ref0 = P3.Cross(ref90,tbm.faces[startFace].normal)
    startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
    forward = seeds['bFor'][i]
    geoline = createGeoLine(startPt, startFace, theta, ref90, forward, maxPathLength = 1000)
    if len(geoline['pts']) < 3:
        geoline = createGeoLine(startPt, startFace, theta, ref90, not forward, maxPathLength = 1000)
    geolines2.append(geoline)

edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on path 45
edge reached on 

In [26]:
i=0
x = startPts[0].x/2 + startPts[1].x/2
y = startPts[0].y/2 + startPts[1].y/2
z = startPts[0].z/2 + startPts[1].z/2

theta = 22
#startPt = P3(x,y,z)
startPt = startPts[i]
ref90 = ref90s[i]
startFace = seeds['faces'][i]
ref0 = P3.Cross(ref90,tbm.faces[startFace].normal)
startVec = rotAxisAngle(ref0,tbm.faces[startFace].normal,theta)
geoline = createGeoLine(startPt, startFace, theta, ref90, forward = False, maxPathLength = 1000)

edge reached on path 22


In [27]:
geolineDouble = createDoubleGeoLine(startPt, startFace, 10, ref90)

edge reached on path 10
edge reached on path 10
PATH FAIL: bad edge reached


In [28]:
CPT = 0.125
thicks = np.zeros(len(F))

for geoline in geolines:
    for f in geoline['faces']:
        if f:
            thicks[f] = thicks[f]+CPT
for geoline in geolines2:
    for f in geoline['faces']:
        if f:
            thicks[f] = thicks[f]+CPT
thicks

array([0.   , 0.5  , 0.5  , ..., 0.5  , 1.75 , 1.875])

In [29]:
ps.init()
geonodes = []
geonodes2 = []
for geoline in geolines:
    geonodes = geonodes + geoline['pts']
for geoline in geolines2:
    geonodes2 = geonodes2 + geoline['pts'] 
print(thicks)
ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
ps_surf.add_scalar_quantity("thickness", thicks, defined_on='faces',enabled = True,cmap='coolwarm')
ps_startline = ps.register_curve_network('startline', P3list2array(startPts), 'line')
#ps_geoline = ps.register_curve_network('geoline', P3list2array(geoline['pts']), 'line')
plotlines =[]
#for i in range(len(geolines)):
#    plotlines.append(P3list2array(geolines[i]['pts']))
ps.register_curve_network('geolines', P3list2array(geonodes), 'line')
ps.register_curve_network('geolines2', P3list2array(geonodes2), 'line')
ps_startPts = ps.register_point_cloud('startPts', P3list2array(startPts))
#ps_SP = ps.register_point_cloud("Start0", P3list2array([start0]))
#ps_SP1 = ps.register_point_cloud("Start1", P3list2array([start1]))
ps_startPts.add_vector_quantity("ref90", P3list2array(ref90s))
ps_startPts.add_vector_quantity("ref0", P3list2array(ref0s))
ps_startPts.add_vector_quantity("startVec", P3list2array(startVecs))
ps.show()

ps.clear_user_callback()

[0.    0.5   0.5   ... 0.5   1.75  1.875]


In [25]:
ps.init()

ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
#ps_SP = ps.register_point_cloud("Start", P3list2array([startPt]))
#ps_SP.add_vector_quantity("ref", np.array([ref90]))


#    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))        

ps.set_user_callback(increment_path)
#ps.set_user_callback(draw_paths)



ps.show()

ps.clear_user_callback()

In [11]:
1158-len(V)

-3853

In [16]:
ps.init()
#for i in range(len(tbm.bars)):
#    pt1 = tbm.bars[i].GetNodeFore(True).p
#    pt2 = tbm.bars[i].GetNodeFore(False).p
#    nodes = np.array([(pt1.x,pt1.y,pt1.z),(pt2.x,pt2.y,pt2.z)])
#    ps_net = ps.register_curve_network('bar'+str(i), nodes, 'line',radius = 0.002, enabled=False)
ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
ps_cloud = ps.register_point_cloud("vertices", V)
#ps_SP = ps.register_point_cloud("Start", P3list2array([startPt]))
#ps_SP.add_vector_quantity("ref", np.array([ref90]))
#for i in range(len(geopaths)):
#    ps_geopath = ps.register_curve_network(str(i), P3list2array(geopaths[i]), 'line')
#    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))
ps_surf.add_vector_quantity("normals", n, defined_on='faces', color=(1, 0.5, 0.5))
ps.show()

In [13]:
n

array([[ 0.47080255, -0.87898719, -0.0756744 ],
       [ 0.45360708, -0.88860863, -0.06793553],
       [ 0.66639608, -0.7434997 , -0.05589595],
       ...,
       [-0.1420244 ,  0.07586244,  0.98695189],
       [-0.44241765, -0.42041126,  0.79216218],
       [-0.43088445, -0.43938151,  0.78821474]])

In [14]:
V

array([[-0.47155201,  0.708579  , -0.199184  ],
       [-0.46079201,  0.73083103, -0.23106299],
       [-0.46033701,  0.68144798, -0.17325   ],
       ...,
       [ 0.46033701,  0.68144798, -0.17325   ],
       [ 0.46079201,  0.73083103, -0.23106299],
       [ 0.47155201,  0.708579  , -0.199184  ]])

In [15]:
F

array([[2688, 2675, 2605],
       [2594, 2605, 2675],
       [2784, 2768, 2692],
       ...,
       [1449, 1454, 1403],
       [1404, 1403, 1376],
       [1372, 1376, 1403]])

In [13]:
ps.init()

ps_surf = ps.register_surface_mesh("my mesh", V, F, smooth_shade=False)
ps_SP = ps.register_point_cloud("Start", P3list2array([startPt]))
ps_SP.add_vector_quantity("ref", np.array([ref90]))
for i in range(len(geopaths)):
    ps_geopath = ps.register_curve_network(str(i), P3list2array(geopaths[i]), 'line')
    ps_geopath.add_scalar_quantity("curvature", np.array(curvs[i]), defined_on='edges', enabled=True,  cmap='coolwarm',vminmax=(-0.1, 0.1))
ps.show()

NameError: name 'startPt' is not defined

In [41]:
seedVec = (0.1,0.2,0)
geopath = []
faces =[]
lam=0.6
bar = tbm.bars[1]
bGoRight = False
ptD = pointOnEdge(1,1-lam)

c = P3(ptD.x-seedVec[0],ptD.y-seedVec[1],ptD.z-seedVec[2])
curv = [1]


geopath.append(c)
for i in range(5000):
    
    if bar != None:
        if bGoRight:
            faces.append(bar.faceright)
        else:
            faces.append(bar.faceleft)
        #print('face: ',faces[-1])
        c, bar, lam, bGoRight = GeoCrossBar(c, bar, lam, bGoRight,False)
        geopath.append(c)
        if faces[-1] is not None:
            CClast = geopath[-1]-geopath[-2]
            curvature = P3.Dot(tbm.faces[faces[-1]].normal,CClast)/P3.Len(tbm.faces[faces[-1]].normal)
            curv.append(curv[-1]*0.75 + curvature*0.25)
            #curv.append( curvature)

    else:
        print('Edge reached')
        break
#print(geopath)

Edge reached
